In [1]:
# encoding: UTF-8
%matplotlib inline

"""
展示如何执行参数优化。
"""

from __future__ import division
from __future__ import print_function

from vnpy.trader.app.ctaStrategy.ctaBacktesting import BacktestingEngine, MINUTE_DB_NAME, OptimizationSetting
from vnpy.trader.app.ctaStrategy.ctaBase import loadContractDetail

import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import time
import sys

def run_optimization(strategy_name, symbol, start_date, end_date,
                    parameter, slippage=1, rate=0.3/10000):
    """优化程序"""
    #--------------------------------------------------
    #载入策略类
    file_name = 'strategy' + strategy_name.replace('Strategy', '')
    file_name = 'vnpy.trader.app.ctaStrategy.strategy.{}'.format(file_name)
    import_file = 'from {} import {}'.format(file_name, strategy_name)
    #如果模块已经载入，删除已经存在的模块
    if file_name in sys.modules.keys():
        del sys.modules[file_name]
        
    exec(import_file)
    strategy_moudle = eval(strategy_name)
    
    #--------------------------------------------------
    #设置回测参数
    start_date = start_date                          #回测起始日期
    end_date = end_date                              #回测终止日期
    symbol = symbol                                  #回测合约
    slippage = slippage                              #滑点为几个跳价
    rate = rate                                      #手续费
    con_d = loadContractDetail(symbol)
    size = con_d['trade_size']                       #合约大小
    price_tick = con_d['price_tick']                 #最小价格变动
    
    #判断参数间是否存在大小关系
    if 'largerParameter' in parameter.keys():
        setting.largerParameter = parameter['largerParameter']
        setting.smallerParameter = parameter['smallerParameter'] 
        del parameter['largerParameter']
        del parameter['smallerParameter']
        
    #设置优化参数与优化目标
    setting = OptimizationSetting()                  # 新建一个优化任务设置对象
    setting.setOptimizeTarget('returnRiskRatio')     # 设置优化排序的目标是策略净盈利
    for p in parameter:
        setting.addParameter(p, parameter[p][0],
                             parameter[p][1], parameter[p][2]) # 增加第优化参数atrMa，起始，结束，步进

    #--------------------------------------------------------------------------------------------------
    # 创建回测引擎
    engine = BacktestingEngine()
    
    # 设置引擎的回测模式为K线
    engine.setBacktestingMode(engine.BAR_MODE)

    # 设置回测用的数据起始日期
    engine.setStartDate(start_date)
    engine.setEndDate(end_date)
    
    # 设置产品相关参数
    engine.setSlippage(slippage * price_tick)     # 滑点
    engine.setRate(rate)                          # 手续费
    engine.setSize(size)                          # 合约大小
    engine.setPriceTick(price_tick)               # 最小价格变动
    
    # 设置使用的历史数据库
    engine.setDatabase(MINUTE_DB_NAME, symbol)
    
    # 跑优化
    # 性能测试环境：I7-3770，主频3.4G, 8核心，内存16G，Windows 7 专业版
    # 测试时还跑着一堆其他的程序，性能仅供参考
    start = time.time()
    
    # 运行单进程优化函数，自动输出结果，耗时：359秒
    # result_list = engine.runOptimization(strategy_moudle, setting)            
    
    # 多进程优化，耗时：89秒
    result_list = engine.runParallelOptimization(strategy_moudle, setting)
    
    #保存结果
    file_name = strategy_moudle.__name__ + '.csv'
    cols = ['parameter','optTarget'] + result_list[0][2].keys()
    s_data = pd.DataFrame(columns=cols)
    
    for r in result_list:
        new_r = dict({'parameter':r[0]}, **{'optTarget':r[1]})
        new_r = dict(new_r, **r[2])
        s_data = s_data.append(pd.Series(new_r), ignore_index=True)
        
    s_data.to_csv(file_name)
    
    #画热力图
    r1 = eval(result_list[0][0])
    keys = list(r1.keys())
    indexs = []
    for k in keys:
        d = []
        for res in result_list:
            dic = eval(res[0])
            d.append(dic[k])
        indexs.append(d)
    
    for i in range(len(keys)-1):
        for j in range(i+1, len(keys)):
            a = list(set(indexs[i]))
            b = list(set(indexs[j]))
            a.sort()
            b.sort()
            data = pd.DataFrame(np.zeros((len(a), len(b))), index=a, columns=b)
            for r in result_list:
                dic = eval(r[0])
                data.loc[dic[keys[i]], dic[keys[j]]] = float(r[1])
            
            f, ax = plt.subplots(figsize = (10, 4))
            cmap = sns.cubehelix_palette(start = 1, rot = 3, gamma=0.8, as_cmap = True)
            sns.heatmap(data, cmap = cmap, linewidths = 0.05, ax = ax)
            ax.set_title('Optimization')
            ax.set_ylabel(keys[i])
            ax.set_xlabel(keys[j])
    plt.show()
    print(u'耗时：%s' %(time.time()-start))

In [ ]:
strategy_name = 'MarketStableStrategy'
start_date = '20160101'
end_date = '20181010'
symbol = 'IF000'
parameter = {'threshold': (0.07, 0.1, 0.01),
            'lossLimit': (0.004, 0.006, 0.001)}

run_optimization(strategy_name, symbol, start_date,
                end_date, parameter)

2019-01-25 10:56:50.400000	------------------------------
2019-01-25 10:56:50.400000	优化结果：
2019-01-25 10:56:50.413000	参数：{'threshold': 0.09999999999999999, 'lossLimit': 0.004}，目标：0.409590328695
2019-01-25 10:56:50.413000	参数：{'threshold': 0.07, 'lossLimit': 0.004}，目标：0.396623718165
2019-01-25 10:56:50.413000	参数：{'threshold': 0.09, 'lossLimit': 0.004}，目标：0.391637184458
2019-01-25 10:56:50.413000	参数：{'threshold': 0.08, 'lossLimit': 0.004}，目标：0.373405444074
2019-01-25 10:56:50.413000	参数：{'threshold': 0.07, 'lossLimit': 0.006}，目标：0.356112973468
2019-01-25 10:56:50.413000	参数：{'threshold': 0.09999999999999999, 'lossLimit': 0.006}，目标：0.340649300043
2019-01-25 10:56:50.413000	参数：{'threshold': 0.09, 'lossLimit': 0.006}，目标：0.329946296242
2019-01-25 10:56:50.413000	参数：{'threshold': 0.09999999999999999, 'lossLimit': 0.005}，目标：0.325944007733
2019-01-25 10:56:50.413000	参数：{'threshold': 0.08, 'lossLimit': 0.006}，目标：0.323981069149
2019-01-25 10:56:50.413000	参数：{'threshold': 0.09, 'lossLimit': 0.005}，目标